In [1]:
import json
import statistics
from collections import Counter
import os 
import shutil


In [2]:
# Load the JSON file
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

# Compute statistics
def compute_keypoint_statistics(data):
    keypoint_counts = [len(model['keypoints']) for model in data]
    
    total_models = len(keypoint_counts)
    total_keypoints = sum(keypoint_counts)
    average_keypoints = statistics.mean(keypoint_counts) if keypoint_counts else 0
    min_keypoints = min(keypoint_counts) if keypoint_counts else 0
    max_keypoints = max(keypoint_counts) if keypoint_counts else 0
    value_counts = Counter(keypoint_counts)
    
    stats = {
        'Total models': total_models,
        'Average keypoints per model': average_keypoints,
        'Minimum keypoints in any model': min_keypoints,
        'Maximum keypoints in any model': max_keypoints,
        'Value counts': value_counts
    }
    
    return stats

In [4]:
filepath = 'guitar.json' 
data = load_json(filepath)
stats = compute_keypoint_statistics(data)

print("Summary Statistics:")
for k, v in stats.items():
    print(f"{k}: {v}")


Summary Statistics:
Total models: 697
Average keypoints per model: 8.464849354375897
Minimum keypoints in any model: 4
Maximum keypoints in any model: 9
Value counts: Counter({9: 587, 5: 79, 7: 14, 8: 12, 6: 4, 4: 1})


In [6]:
# Paths
mesh_dir = "Guitars/All"  # your source directory containing .ply files
output_dir = "Guitars/9_points/"  # directory to store the filtered files
annotation_file = "guitar.json"  # your annotation json
n = 9

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Load annotations
with open(annotation_file, "r") as f:
    annotations = json.load(f)

# Build a lookup for model_ids with exactly 9 keypoints
model_ids_with_n_kp = []
for entry in annotations:
    model_id = entry["model_id"]
    keypoints = entry["keypoints"]
    if len(keypoints) == n:
        model_ids_with_n_kp.append(model_id)

print(f"Found {len(model_ids_with_n_kp)} models with {n} keypoints.")

# Copy matching .ply files to output directory
for model_id in model_ids_with_n_kp:
    src_file = os.path.join(mesh_dir, f"{model_id}.ply")
    dst_file = os.path.join(output_dir, f"{model_id}.ply")
    
    if os.path.exists(src_file):
        shutil.copyfile(src_file, dst_file)
    else:
        print(f"Warning: {src_file} not found!")

print("Done!")


Found 587 models with 9 keypoints.
Done!


In [3]:
# Function that filters and writes filtered JSON data
def filter_and_write_models(filepath, target_count, output_path):
    data = load_json(filepath)
    selected_models = [model for model in data if len(model['keypoints']) == target_count]
    with open(output_path, 'w') as f:
        json.dump(selected_models, f, indent=2)
    print(f"Saved {len(selected_models)} models with {target_count} keypoints to {output_path}")

In [17]:
filter_and_write_models("guitar.json", 9, "guitar_9.json")

Saved 587 models with 9 keypoints to guitar_9.json


### Ground Truth Keypoints for Model

In [4]:
def load_annotations(annotation_file):
    with open(annotation_file, "r") as f:
        annotations = json.load(f)
    
    model_id_to_keypoints = {}
    for entry in annotations:
        model_id = entry["model_id"]
        keypoints = entry["keypoints"]
        model_id_to_keypoints[model_id] = [kp["xyz"] for kp in keypoints]
    
    return model_id_to_keypoints


def get_keypoints_for_model(model_id, annotation_file):
    model_id_to_keypoints = load_annotations(annotation_file)
    
    if model_id in model_id_to_keypoints:
        return model_id_to_keypoints[model_id]
    else:
        print(f"Model ID {model_id} not found in annotations.")
        return None

In [8]:
annotation_file = "guitar.json"  # Path to your annotation file
model_id_example = "ef1c22bd3b74953689f0379846507dd3"
keypoints = get_keypoints_for_model(model_id_example, annotation_file)
if keypoints:
    for i, kp in enumerate(keypoints):
        print(f"{kp[0]:.3f} {kp[1]:.3f} {kp[2]:.3f}")


0.024 0.417 0.008
0.013 -0.169 -0.002
0.102 -0.140 0.015
-0.072 -0.157 0.010
0.143 -0.478 0.013
-0.114 -0.492 0.012
0.012 -0.532 0.015
0.111 -0.326 0.014
-0.085 -0.316 0.012


### Test Train Split

In [5]:
import random

In [6]:
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)

def split_dataset(filepath, train_ratio=0.8, seed=42):
    data = load_json(filepath)
    random.seed(seed)
    random.shuffle(data)
    
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]
    
    save_json(train_data, 'guitar_train.json')
    save_json(test_data, 'guitar_test.json')
    
    print(f"Dataset split completed.")
    print(f"Total samples: {len(data)}")
    print(f"Training samples: {len(train_data)}")
    print(f"Testing samples: {len(test_data)}")



In [7]:
filepath = 'guitar_9.json' 
split_dataset(filepath)

Dataset split completed.
Total samples: 587
Training samples: 469
Testing samples: 118
